In [1]:
%load_ext autoreload
%autoreload 2


# Imports

In [2]:
import duckdb
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
#from scipy import signal
#import mlflow
from rcssim import rcs_sim as rcs
import plotly.express as px
from typing import Tuple
import vegafusion as vf
#from joypy import joyplot
from joblib import Parallel, delayed
from itertools import combinations
import json


In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, balanced_accuracy_score, make_scorer

In [62]:
import ray
num_GB_to_allocate = 50
memory_allocation_in_bytes = num_GB_to_allocate * 1024**3
allocate_memory = False
base_path = '/media/shortterm_ssd/Clay/ray_spill/'
# The object_store_memory flag tells ray how much memory to allocate to the object store (stored in RAM, as a RAM disk. The object store is written to tmpfs, mounted on /dev/shm),
# before spill over to disk occurs. One should consider using the allocate_memory flag if RAM is tight and spillover to disk is desired and controlled.
if allocate_memory:
    ray.init(object_store_memory=memory_allocation_in_bytes,
             _system_config={
                            "max_io_workers": 4,  # More IO workers for parallelism.
                                "object_spilling_config": json.dumps(
                                    {
                                        "type": "filesystem",
                                        "params": {
                                            # Multiple directories can be specified to distribute
                                            # IO across multiple mounted physical devices.
                                            "directory_path": [
                                            base_path+"/tmp/spill",
                                            base_path+"/tmp/spill_1",
                                            base_path+"/tmp/spill_2",
                                            ]
                                        },
                                    }
                            )
             })
else:
    # I don't think spill-over is working correctly with current config... Raylet was killing jobs because of 'out of memory' errors.
    ray.init(_system_config={
                            "max_io_workers": 4,  # More IO workers for parallelism.
                            "object_spilling_config": json.dumps(
                                    {
                                        "type": "filesystem",
                                        "params": {
                                            # Multiple directories can be specified to distribute
                                            # IO across multiple mounted physical devices.
                                            "directory_path": [
                                            base_path+"/tmp/spill",
                                            base_path+"/tmp/spill_1",
                                            base_path+"/tmp/spill_2",
                                            ]
                                        },
                                    }
                            )
             })

2023-04-04 13:37:38,846	INFO worker.py:1553 -- Started a local Ray instance.


In [5]:
import plotly.io as pio
alt.data_transformers.disable_max_rows()
dataspell = False
use_data_server = False # Use data server if you want to use the Vega Editor window
use_browser_for_plotly=False
remote = False

# The below commands allow for Altair to plot large data sets.

if dataspell:
    if use_data_server:
        alt.data_transformers.enable('data_server') # <- Exports to temporary localHost URL
        print('Data Server Enabled')
    else:
        vf.enable()
        print('VegaFusion Enabled')
    if use_browser_for_plotly:
        pio.renderers.default = 'browser'
    print("DataSpell Setup")
else:
    if not remote:
        # DO NOT RUN BELOW IN REMOTE VSCODE.
        if use_data_server:
            alt.data_transformers.enable('data_server') # <- Exports to temporary localHost URL
            print('Data Server Enabled')
        else:
            # Vegafusion allows importing polars dataframes into Altair charts directly, so should use this widget whenever possible.
            vf.enable_widget(data_dir='/media/shortterm_ssd/Clay/altairdata/')
            print('VegaFusion Widget Enabled')

    if use_browser_for_plotly:
        pio.renderers.default = 'browser'
        print("Plotly Browser Enabled")

    from plotly_resampler import register_plotly_resampler, unregister_plotly_resampler
    register_plotly_resampler(mode='auto')
    print("Plotly Resampler Enabled")
    print("VS Code")
'''
Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']
'''
pio.templates.default = "plotly_white"


VegaFusion Widget Enabled
Plotly Resampler Enabled
VS Code


In [6]:
# Test if altair displays
tmp_df = pd.DataFrame({'A': np.arange(50), 'B': np.arange(50,100)})
alt.Chart(tmp_df).mark_point().encode(
    x='A:Q',
    y='B:Q',
).interactive()

VegaFusionWidget(spec='{\n  "config": {\n    "view": {\n      "continuousWidth": 400,\n      "continuousHeight…

In [7]:
# Test if plotly displays
px.box(pd.DataFrame({'A': np.arange(50), 'B': np.arange(50,100)}).melt(value_vars=['A', 'B']), x='variable', y='value')

FigureWidgetResampler({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=%{x}<…

(hyperparameter_search_wrapper pid=1321736) Analayzing: TD_BG, TD_key2, TD_key3


2023-04-04 12:22:25,084	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::hyperparameter_search_wrapper() (pid=1331028, ip=10.37.129.11)
  File "/tmp/ipykernel_1320946/3663473140.py", line 9, in hyperparameter_search_wrapper
  File "/tmp/ipykernel_1320946/2533931492.py", line 27, in hyperparameter_search_pipeline
IndexError: index 3 is out of bounds for axis 0 with size 3


(hyperparameter_search_wrapper pid=1330970) Analyzing: TD_key2, TD_key3


2023-04-04 12:22:47,090	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::hyperparameter_search_wrapper() (pid=1330970, ip=10.37.129.11)
  File "/tmp/ipykernel_1320946/3663473140.py", line 9, in hyperparameter_search_wrapper
  File "/tmp/ipykernel_1320946/2533931492.py", line 27, in hyperparameter_search_pipeline
IndexError: index 3 is out of bounds for axis 0 with size 3


...

2023-04-04 13:29:46,887	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::hyperparameter_search_wrapper() (pid=1330959, ip=10.37.129.11)
  File "/tmp/ipykernel_1320946/3663473140.py", line 9, in hyperparameter_search_wrapper
  File "/tmp/ipykernel_1320946/3079146302.py", line 38, in hyperparameter_search_pipeline
  File "/tmp/ipykernel_1320946/1746186864.py", line 44, in add_simulated_ffts
  File "/tmp/ipykernel_1320946/1746186864.py", line 39, in fft_sim_row
IndexError: list index out of range


(hyperparameter_search_wrapper pid=1330970) ['TD_key2', 'TD_key3', 'TD_count', 'TD_null']
(hyperparameter_search_wrapper pid=1330970) (4, 181393, 512)
(hyperparameter_search_wrapper pid=1330970) (181393, 5)
(hyperparameter_search_wrapper pid=1330970) [231, 234]
(hyperparameter_search_wrapper pid=1330970) (4, 181393, 512)
(hyperparameter_search_wrapper pid=1330970) (181393, 5)
(hyperparameter_search_wrapper pid=1330970) [231, 234]
(hyperparameter_search_wrapper pid=1330970) (4, 181393, 512)
(hyperparameter_search_wrapper pid=1330970) (181393, 5)
(hyperparameter_search_wrapper pid=1330970) [231, 234]


2023-04-04 13:30:18,896	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::hyperparameter_search_wrapper() (pid=1330970, ip=10.37.129.11)
  File "/tmp/ipykernel_1320946/3663473140.py", line 9, in hyperparameter_search_wrapper
  File "/tmp/ipykernel_1320946/3079146302.py", line 38, in hyperparameter_search_pipeline
  File "/tmp/ipykernel_1320946/1746186864.py", line 44, in add_simulated_ffts
  File "/tmp/ipykernel_1320946/1746186864.py", line 39, in fft_sim_row
IndexError: list index out of range


(hyperparameter_search_wrapper pid=1331024) Executing Device:  09L


(raylet) [2023-04-04 13:37:00,485 E 1330841 1330841] (raylet) node_manager.cc:3040: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 569141cbc5ebfde1dd31aa81dd7722ee96f640efb88b175823a975ce, IP: 10.37.129.11) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.37.129.11`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
2023-04-04 13:37:00,999	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): r

# Extract relevant data

In [9]:
# TODO: Put this into script so can be accessed as a module


In [9]:
con = duckdb.connect(database='/media/shortterm_ssd/Clay/databases/duckdb/rcs-db.duckdb', read_only=True)

In [8]:
def get_device_as_pl_df(device, db_con, lazy=False):
    """
    Accesses duckdb database and returns columns of interest, labeled reasonably with session identifiers cast as categoricals.
    :param device: device name (str) (e.g. '02L')
    :param db_con: duckdb connection object
    :return: polars Dataframe
    """
    db_con.sql("SET TIMEZONE = 'America/Los_Angeles'")
    if lazy:
        return db_con.sql(f"select DerivedTime, columns('localTime'), columns('^Session|TD_|Power_Band'), SleepStage from overnight.r{device}").pl().lazy().with_columns(
            pl.col('^Session.*$').cast(pl.Categorical)
        )
    else:
        return db_con.sql(f"select DerivedTime, columns('localTime'), columns('^Session|TD_|Power_Band'), SleepStage from overnight.r{device}").pl().with_columns(
            pl.col('^Session.*$').cast(pl.Categorical)
        )


In [9]:
def get_device_as_pd_df(device, db_con):
    """
    Accesses duckdb database and returns columns of interest, labeled reasonably with session identifiers cast as categoricals.
    :param device: device name (str) (e.g. '02L')
    :param db_con: duckdb connection object
    :return: polars Dataframe
    """
    return db_con.execute(f"select DerivedTime, columns('localTime'), columns('^Session|TD_|Power_Band'), SleepStage from overnight.r{device}").df()


# RCS_SIM Functions

In [10]:
def rcssim_wrapper(td_data, times, settings, gain) -> Tuple[np.ndarray, int]:
    """
    This function takes in polars lists of times (DerivedTimes) and timedomain data (td_data), and returns an estimate of the embedded RCS fft vector.
    params:
    times: (pl.list of floats: unix timestamps) 1xn vector of DerivedTimes
    td_data: (pl.list of floats: millivolts) 1xn vector of time domain data
    settings: (dict) settings for device
    returns: data_fft_pl: (pl.list of floats) Estimate of embedded FFT vector(s) (size: mxn) corresponding to td_data
             t_pb: (float: unix timestamp) DerivedTime unix timestamp of FFT vector
    """
    hann_win = rcs.create_hann_window(settings['fft_size'][0], percent=100)
    # times_np = times.to_numpy(zero_copy_only=True)

    # td_np = rcs.transform_mv_to_rcs(td_data.to_numpy(zero_copy_only=True), gain)
    td_np = rcs.transform_mv_to_rcs(td_data, gain)

    data_fft, t_pb = rcs.td_to_fft(td_np, times,
                                   settings['samplingRate'][0],
                                   settings['fft_size'][0], settings['fft_interval'][0],
                                   hann_win, interp_drops=False, output_in_mv=False, shift_timestamps_up_by_one_ind=True)
    data_fft_out = rcs.fft_to_pb(data_fft, settings['samplingRate'][0], settings['fft_size'][0],
                                 settings['fft_bandFormationConfig'][0],
                                 input_is_mv=False)
    return data_fft_out, t_pb


In [11]:
def get_gains_from_settings_dict(settings, sessions, identity_col, device, db_con) -> np.ndarray:
    """

    :param settings:
    :param sessions:
    :param identity_col:
    :param device:
    :param db_con:
    :return:
    """
    td_settings = db_con.execute(f"Select columns('^chan'), columns('^gain') from r{device}.TDSettings where {identity_col} in {*sessions,}").df().drop_duplicates()
    if len(td_settings) > 1:
        print(f"WARNING: MULTIPLE TIME DOMAIN SETTINGS FOUND for {device} sessions {sessions}. Using the first unique values.")
    gains = []
    for i in range(1,len(td_settings.columns)+1):
        if (f"chan{i}" in td_settings.columns) and not ('Disabled' in td_settings[f"chan{i}"][0]):
            gains.append(td_settings[f"gain_{i}"][0])
    return np.array(gains)


In [12]:
def get_settings_for_pb_calcs(device, db_con, session_nums, identity_col):
    """
    Get the relevant settings to run rcs_sim package functions:
        rcs.create_hann_window
        rcs.transform_mv_to_rcs
        rcs.td_to_fft
        rcs.fft_to_pb

    :param device:
    :param db_con:
    :param session_nums: list of SessionIdentity values to pull settings from
    :return:
    """
    # TODO: FIX THIS FUNCTION TO ALLOW FOR identity_col = 'Session#'

    td_settings = db_con.execute(f"Select {identity_col}, samplingRate, columns('^gain') from r{device}.TDSettings where {identity_col} in {*session_nums,}").df()
    fft_settings = db_con.execute(f"Select {identity_col}, fft_bandFormationConfig, fft_interval, fft_size, fft_windowLoad, fft_numBins, fft_binWidth, columns('^Power_Band') from r{device}.FftAndPowerSettings where SessionIdentity in {*session_nums,}").df()
    settings_df = pd.concat([td_settings, fft_settings], axis=1, join='inner')
    settings_df['fft_bandFormationConfig'] = settings_df['fft_bandFormationConfig'].str.extract('(\d+)').astype(int)
    if not (np.sum(np.unique(settings_df.loc[:, settings_df.columns!=identity_col].nunique().values)) == 1):
        print("WARNING: Session numbers provided have different settings.")
        print('Settings dataframe:')
        print(settings_df)

    return settings_df.loc[:, settings_df.columns!=identity_col].drop_duplicates().to_dict('list')

## Model Pipeline Development


PIPELINE (each step will be an aggregate of functions)

  1. Collect Patient Data
  2. Transform into TD Windows
  For Shift value
  3. Collect FFTs via RCS_SIM package
  4. COLLECT PBs

Additionally,
  1. Split FFT vectors into sections based on correlation (or mutual information) values, magnitude needs to be above threshold
  1. Create Grid of hyperparameters
  2. Train and cross validate each of the below models on data collected above\
    *LDA\
    *2-step LDA\
    *Decision Tree

Be careful to keep track of how data was partitioned in cross-validation, so that models can easily be recreated.

Save Grid and Classification Scores in pl Dataframe, and export to parquet or DuckDB (or both)

After the best hyperparameters have been identified, use ALL data to train a final model for production.


_____________________________________________________________________

In [13]:
sleep_stage_mapping = {2: 1, 3: 1, 4: 0, 5: 0, 6: 0}


## Start of pipeline

In [14]:
def chunk_df_by_timesegment_deprecated(df, interval='1s', period='2s', sample_rate=500, align_with_PB_outputs=False):
    """
    Chunk a dataframe into smaller dataframes based on a time interval and period.
    The period is the length of the time segment, the interval is the time between the start of each time segment.
    
    Parameters:
    df (DataFrame): The dataframe to be chunked
    interval (str): The time interval between the start of each time segment. Default is '1s'
    period (str): The length of each time segment. Default is '2s'
    align_with_PB_outputs (bool): If True, the time segments will be aligned with the Power Band outputs. Default is False.
    """
    # TODO: Remove hardcoding of 'SleepStage', should refer to it as a variable
    if align_with_PB_outputs:
        df_pb_count = df.join(
            df.filter(pl.col('Power_Band8').is_not_null()).select(
                'DerivedTime').with_row_count(),
            on='DerivedTime', how='left').with_columns(pl.col('row_nr').fill_null(strategy='backward')).rename({'row_nr': 'PB_count'})

        df_pb_count = df_pb_count.with_columns([
            pl.when( (pl.col('PB_count') % 2) == 0).then(pl.lit(None)).otherwise(pl.col('PB_count')).fill_null(strategy='backward').alias('PB_count_odd'),
            pl.when( (pl.col('PB_count') % 2) == 1).then(pl.lit(None)).otherwise(pl.col('PB_count')).fill_null(strategy='backward').alias('PB_count_even')
        ])

        df_pb_count = df_pb_count.groupby(['SleepStage', 'PB_count_even']).agg(
            [
                pl.col('DerivedTime'),
                pl.col('^Power_Band.*$').drop_nulls().first(),
                pl.col('TD_BG'),
                pl.col('TD_key2'),
                pl.col('TD_key3'),
                pl.col('TD_BG').count().alias('TD_count')
            ]).rename({'PB_count_even': 'PB_ind'}).vstack(
                df_pb_count.groupby(['SleepStage', 'PB_count_odd']).agg(
                    [
                        pl.col('DerivedTime'),
                        pl.col('^Power_Band.*$').drop_nulls().first(),
                        pl.col('TD_BG'),
                        pl.col('TD_key2'),
                        pl.col('TD_key3'),
                        pl.col('TD_BG').count().alias('TD_count')
                    ]).rename({'PB_count_odd': 'PB_ind'})
        ).select(pl.all().shrink_dtype()).rechunk()

        df_chunked = df_pb_count
    else:
        df_grouped = df.sort('localTime').groupby_dynamic('localTime', every=interval, period=period, by=['SessionIdentity', 'SleepStage']).agg([
            pl.col('DerivedTime'),
            pl.col('^Power_Band.*$').drop_nulls().first(),
            pl.col('TD_BG'),
            pl.col('TD_key2'),
            pl.col('TD_key3'),
            pl.col('TD_BG').count().alias('TD_count')]).select(pl.all().shrink_dtype())

        df_grouped = df_grouped.with_columns(
                    pl.col('TD_BG').arr.eval(pl.element().is_null().any()).alias('TD_null')
                ).filter((pl.col('TD_count') == int(period[0]) * sample_rate ) &
                        (pl.col('TD_null').arr.contains(False))
                        )
        df_chunked = df_grouped
    return df_chunked

In [54]:
def chunk_df_by_timesegment(df, interval='1s', period='2s', sample_rate=500, align_with_PB_outputs=False, td_columns=['TD_BG', 'TD_key2', 'TD_key3']):
    """
    Chunk a dataframe into smaller dataframes based on a time interval and period.
    The period is the length of the time segment, the interval is the time between the start of each time segment.
    
    Parameters:
    df (DataFrame): The dataframe to be chunked
    interval (str): The time interval between the start of each time segment. Default is '1s'
    period (str): The length of each time segment. Default is '2s'
    align_with_PB_outputs (bool): If True, the time segments will be aligned with the Power Band outputs. Default is False.
    """
    td_cols = [col for col in df.columns if col in td_columns]
    # TODO: Remove hardcoding of 'SleepStage', should refer to it as a variable
    if align_with_PB_outputs:
        df_pb_count = df.join(
            df.filter(pl.col('Power_Band8').is_not_null()).select(
                'DerivedTime').with_row_count(),
            on='DerivedTime', how='left').with_columns(pl.col('row_nr').fill_null(strategy='backward')).rename({'row_nr': 'PB_count'})

        df_pb_count = df_pb_count.with_columns([
            pl.when( (pl.col('PB_count') % 2) == 0).then(pl.lit(None)).otherwise(pl.col('PB_count')).fill_null(strategy='backward').alias('PB_count_odd'),
            pl.when( (pl.col('PB_count') % 2) == 1).then(pl.lit(None)).otherwise(pl.col('PB_count')).fill_null(strategy='backward').alias('PB_count_even')
        ])

        df_pb_count = df_pb_count.groupby(['SleepStage', 'PB_count_even']).agg(
            [
                pl.col('DerivedTime'),
                pl.col('^Power_Band.*$').drop_nulls().first(),
                pl.col('^TD_.*$'),
                pl.col(td_cols[0]).count().alias('TD_count')
            ]).rename({'PB_count_even': 'PB_ind'}).vstack(
                df_pb_count.groupby(['SleepStage', 'PB_count_odd']).agg(
                    [
                        pl.col('DerivedTime'),
                        pl.col('^Power_Band.*$').drop_nulls().first(),
                        pl.col('^TD_.*$'),
                pl.col(td_cols[0]).count().alias('TD_count')
                    ]).rename({'PB_count_odd': 'PB_ind'})
        ).select(pl.all().shrink_dtype()).rechunk()

        df_chunked = df_pb_count
    else:
        df_grouped = df.sort('localTime').groupby_dynamic('localTime', every=interval, period=period, by=['SessionIdentity', 'SleepStage']).agg([
            pl.col('DerivedTime'),
            pl.col('^Power_Band.*$').drop_nulls().first(),
            pl.col('^TD_.*$'),
                pl.col(td_cols[0]).count().alias('TD_count')]).select(pl.all().shrink_dtype())

        df_grouped = df_grouped.with_columns(
                    pl.col(td_cols[0]).arr.eval(pl.element().is_null().any()).alias('TD_null')
                ).filter((pl.col('TD_count') == int(period[0]) * sample_rate ) &
                        (pl.col('TD_null').arr.contains(False))
                        )
        df_chunked = df_grouped
    return df_chunked

In [16]:
def add_simulated_ffts_deprecated(df_chunked, settings, gains, shift=None):    
    """
    Add simulated FFTs to a dataframe of time segments. This function calls the rcs_sim package to simulate RC+S outputs
    
    parameters:
    df_chunked (pl.DataFrame): The dataframe of time segments, as output by chunk_df_by_timesegment
    settings (dict): The settings dictionary for the simulation (usually replicating the settings used to generate the original FFTs from an RCS session)
    gains (list): The amp gains to use for the simulation
    fft_subset_inds (list): The indices of the FFTs to use for the simulation. Default is [2,120]

    returns:
    df_chunked (pl.DataFrame): The dataframe of time segments with simulated FFTs added as additional columns. Each time domain channel has its own set of simulated FFTs.
    """

    # Select the time domain channels and timestamps to use for the simulating FFTs
    td_np = df_chunked.select([
        pl.col('DerivedTime'),
        pl.col('TD_BG'),
        pl.col('TD_key2'),
        pl.col('TD_key3')
    ]).collect().to_numpy()

    fft_arr_BG = np.zeros((td_np.shape[0], settings['fft_numBins'][0]))
    fft_arr_2 = np.zeros((td_np.shape[0], settings['fft_numBins'][0]))
    fft_arr_3 = np.zeros((td_np.shape[0], settings['fft_numBins'][0]))

    sim_settings = settings.copy()
    if shift:
        sim_settings['fft_bandFormationConfig'] = [shift]


    # Could try to use numba to speed up the loop, or use 'pl.apply' on the dataframe
    # Simulate FFTs for each time segment, one FFT period at a time

    # This is a horrible way to access gains. Need to make parameterize it and verify which subcortical gain is being used...
    def fft_sim_row(i):
        fft_arr_BG[i], _ = rcssim_wrapper(td_np[i,1], td_np[i,0], sim_settings, gains[0])
        fft_arr_2[i], _ = rcssim_wrapper(td_np[i,2],td_np[i,0],  sim_settings, gains[1])
        fft_arr_3[i], _ = rcssim_wrapper(td_np[i,3],td_np[i,0],  sim_settings, gains[2])


    #@njit(parallel=True)
    for i in range(td_np.shape[0]):
        fft_sim_row(i)

    # df_chunked = df_chunked.hstack(
    # pl.DataFrame({'fft_BG': fft_arr_BG,
    #                 'fft_key2': fft_arr_2,
    #                 'fft_key3': fft_arr_3})
    # )

    # Join the simulated FFTs to the dataframe
    df_chunked = df_chunked.with_row_count().join(
                            pl.LazyFrame({'fft_BG': fft_arr_BG,
                                            'fft_key2': fft_arr_2,
                                            'fft_key3': fft_arr_3}).with_row_count(), 
                            on='row_nr', how='left')

    return df_chunked

In [56]:
def add_simulated_ffts(df_chunked, settings, gains, shift=None, td_columns=['TD_BG', 'TD_key2', 'TD_key3']):    
    """
    Add simulated FFTs to a dataframe of time segments. This function calls the rcs_sim package to simulate RC+S outputs
    
    parameters:
    df_chunked (pl.DataFrame): The dataframe of time segments, as output by chunk_df_by_timesegment
    settings (dict): The settings dictionary for the simulation (usually replicating the settings used to generate the original FFTs from an RCS session)
    gains (list): The amp gains to use for the simulation
    fft_subset_inds (list): The indices of the FFTs to use for the simulation. Default is [2,120]

    returns:
    df_chunked (pl.DataFrame): The dataframe of time segments with simulated FFTs added as additional columns. Each time domain channel has its own set of simulated FFTs.
    """

    # Select the time domain channels and timestamps to use for the simulating FFTs
    td_np = df_chunked.select([
        pl.col('DerivedTime'),
        pl.col('^TD_.*$')
    ]).collect().to_numpy()

    td_cols = [col for col in df_chunked.columns if col in td_columns]

    print(td_cols)

    fft_arr = np.zeros((len(td_cols), td_np.shape[0], settings['fft_numBins'][0]))

    sim_settings = settings.copy()
    if shift:
        sim_settings['fft_bandFormationConfig'] = [shift]


    # Could try to use numba to speed up the loop, or use 'pl.apply' on the dataframe
    # Simulate FFTs for each time segment, one FFT period at a time
    def fft_sim_row(i):
        for j in range(len(td_cols)):
            fft_arr[j,i], _ = rcssim_wrapper(td_np[i,j+1], td_np[i,0], sim_settings, gains[j])


    #@njit(parallel=True)
    for i in range(td_np.shape[0]):
        fft_sim_row(i)

    # Join the simulated FFTs to the dataframe
    df_chunked = df_chunked.with_row_count().join(
                            pl.LazyFrame({f'fft_{i}': fft_arr[i] for i in range(len(td_cols))}).with_row_count(), 
                            on='row_nr', how='left')

    return df_chunked

In [18]:
def expand_fft_arrays_with_subset_deprecated(df, fft_ind_start, vec_length):
    """
    Take a subset of each Time Domain Channels FFT simulations, and expand each fft bin into it's own column

    parameters:
    df (pl.DataFrame): The dataframe of time segments with simulated FFTs added as additional columns. Each time domain channel has its own set of simulated FFTs.
    fft_ind_start (int): The index of the first FFT bin to use
    vec_length (int): The number of FFT bins to use when calculating the fft subset

    returns:
    df (pl.DataFrame): The dataframe of time segments with simulated FFTs bins added as individual columns
    """
    return (df.with_columns([
                        pl.col('^fft_.*$').arr.slice(fft_ind_start, vec_length)
                    ])
                    .with_columns(
                        pl.col('fft_BG').arr.concat([pl.col('fft_key2'), pl.col('fft_key3')]).arr.to_struct().alias('fft_vec')
                    )
                    .unnest('fft_vec')
                    )

In [19]:
def expand_fft_arrays_with_subset(df, fft_ind_start, vec_length):
    """
    Take a subset of each Time Domain Channels FFT simulations, and expand each fft bin into it's own column

    parameters:
    df (pl.DataFrame): The dataframe of time segments with simulated FFTs added as additional columns. Each time domain channel has its own set of simulated FFTs.
    fft_ind_start (int): The index of the first FFT bin to use
    vec_length (int): The number of FFT bins to use when calculating the fft subset

    returns:
    df (pl.DataFrame): The dataframe of time segments with simulated FFTs bins added as individual columns
    """
    fft_cols = [col for col in df.columns if 'fft' in col]
    return (df.with_columns([
                        pl.col('^fft_.*$').arr.slice(fft_ind_start, vec_length)
                    ])
                    .with_columns(
                        pl.col(fft_cols[0]).arr.concat(pl.col(fft_cols[1:])).arr.to_struct().alias('fft_vec')
                    )
                    .unnest('fft_vec')
                    )

In [64]:
def get_fft_corrs(df_chunked, fft_subset_inds=[2,120], label_col='SleepStage', td_columns=['TD_BG', 'TD_key2', 'TD_key3']) -> np.ndarray:
    """
    Calculate the correlation between each FFT bin and the label columns (SleepStage, or other)

    parameters:
    df_chunked (pl.DataFrame): The dataframe of time segments with simulated FFTs added as additional columns. Each time domain channel has its own set of simulated FFTs.
    fft_subset_inds (list): The indices of the subset of each FFT output to use for the simulation. Default is [2,120]
    label_col (str): The column to use for the label, for which each frequency bin will be compared to in a pearson correlation measure. Default is 'SleepStage'

    returns:
    df_fft_vec (pl.DataFrame): The dataframe with the desired subset of simulated FFTs bins for each FFT interval added as individual columns
    fft_corrs (np.ndarray): The correlation between each FFT bin and the label column
    """

    fft_ind_start = fft_subset_inds[0]
    vec_length = fft_subset_inds[1] - fft_subset_inds[0]

    df_fft_vec = expand_fft_arrays_with_subset(df_chunked, fft_ind_start=fft_ind_start, vec_length=vec_length)

    # Could convert fft_corrs to be a pl.dataframe with discord trick by @ms. This would allow it to be lazy and save on memory.
    fft_corrs = df_fft_vec.select([
                        pl.corr(f"field_{i}", pl.col(label_col).cast(pl.Float64), method='pearson').alias(f"field_{i}_corr") for i in range(vec_length*len(td_columns))
                    ]).to_numpy().squeeze()
    
    return df_fft_vec, fft_corrs

In [21]:
def get_train_data_by_corr_threshold(df, fft_corrs, corr_threshold, label_col='SleepStageBinary'):
    """
    Reduce features for powerband selection, by selecting the FFT bins that have a correlation above the threshold

    parameters:
    df (pl.DataFrame): The dataframe of time segments (i.e. individual fft windows are single rows) with simulated FFTs added as additional columns.
    corr_threshold (float): The threshold for the correlation between the FFT bins and the label column. Only FFT bins with a correlation above this threshold will kept for future feature selection.
    label_col (str): The column to use for the label, for which each frequency bin will be compared to in a pearson correlation measure. Default is 'SleepStageBinary'
    """
    X = df.select(pl.col("^field_.*$")).to_numpy()
    feature_group = np.argwhere(np.abs(fft_corrs) > corr_threshold)
    X = X[:, np.where(np.abs(fft_corrs) > corr_threshold)[0]]
    y = df.select(pl.col(label_col)).to_numpy().squeeze()
    return feature_group, X, y

In [22]:
def get_PB_combinations_deprecated(sfs, feature_group, max_clusters=8):
    PB_groupings = []
    for i in range(2, max_clusters+1): 
        kmeans = KMeans(n_clusters=i, random_state=0).fit(sfs.get_support(indices=True)[:, np.newaxis])

        pbs = [[feature_group[np.argwhere(kmeans.labels_ == j).squeeze()].min(), 
                feature_group[np.argwhere(kmeans.labels_ == j).squeeze()].max()] for j in range(i)]
        
        cluster_df = pd.DataFrame({'x':sfs.get_support(indices=True), 'y':np.zeros(sfs.get_support(indices=True).shape[0]), 'color':kmeans.labels_})
        if i == 2:
            chart = alt.Chart(cluster_df).mark_point().encode(x='x', y='y', color=alt.Color('color', scale=alt.Scale(scheme='category20b')))
        else:
            chart &= alt.Chart(cluster_df).mark_point().encode(x='x', y='y', color=alt.Color('color', scale=alt.Scale(scheme='category20b')))
        
        PB_groupings.append(pbs)
    
    pb_combos = []
    for pbs in PB_groupings:
        if len(pbs) <= 4:
            pb_combos.append(pbs)
        else:
            pb_combos.append(list(combinations(pbs, 4)))
            
    return pb_combos, chart

In [23]:
def get_PB_combinations(features, max_clusters=8):
    """
    Takes in the features and returns the possible powerband combinations. Features are fft indices as determined by SequentialFeatureSelector.

    parameters:
    features (np.ndarray): The fft indices that were selected by SequentialFeatureSelector. They chould correspond to the indicies within the entire fft vector (i.e. the three channels fft outputs horizontally concatenated), 
        not the subset of the FFTs that was used for feature selection.
    max_clusters (int): The maximum number of possible powerbands to consider for selection. Default is 8.

    returns:
    pb_combos (list): A list of lists of lists. The first list is the combination of powerbands for the corresponding cluster amount, the second list is the individual powerbands, 
    and the third list is the start and end indices of the powerband.
    chart (altair.Chart): An interactive chart of the powerband combinations. The x-axis is the fft indices, the y-axis is irrelevant, and the color is the cluster number.
    """
    PB_groupings = []
    for i in range(2, max_clusters+1): 
        kmeans = KMeans(n_clusters=i, random_state=0).fit(features[:, np.newaxis])

        # Extract first and last index of each cluster, to be used as powerband boundaries
        pbs = [[features[np.argwhere(kmeans.labels_ == j).squeeze()].min(), 
                features[np.argwhere(kmeans.labels_ == j).squeeze()].max()] for j in range(i)]
        
        cluster_df = pd.DataFrame({'x':features, 'y':np.zeros(features.shape[0]), 'color':kmeans.labels_})
        base = alt.Chart(cluster_df).mark_point().encode(x='x', y='y', color=alt.Color('color', scale=alt.Scale(scheme='category20b'))).interactive()
        if i == 2:
            chart = base
        else:
            chart &= base
        
        PB_groupings.append(pbs)
    
    pb_combos = []
    for pbs in PB_groupings:
        if len(pbs) <= 4:
            pb_combos.append(pbs)
        else:
            # Get all combinations of 4 powerbands chosen from the possible powerbands in that cluster (e.g. if num clusters is 8, add all combinations of 4 powerbands from the 8 powerbands)
            pb_combos.append(list(combinations(pbs, 4)))
            
    return pb_combos, chart

In [24]:
def recover_original_feature_inds(feature_list, fft_subset_inds, fft_length):
    """
    Recover the original fft indices (i.e. the indices of the fft bins from the FFT vector that corresponds to the concatenated simulations of each TD channel) from the feature list. This typically refers to putative powerband edges. 
    Recall the features were collected from the fft subset indices.

    parameters:
    feature_list (list): The list of features.
    fft_subset_inds (list): The start and end indices of the fft subset that was used for feature selection.
    fft_length (int): The length of the simulated fft vector for each TD channel (e.g. 512 for an FFT window of 1024 on a 500 TD sampling rate).

    returns:
    list: The list of original fft indices.
    """
    vec_length = fft_subset_inds[1] - fft_subset_inds[0]
    vec_start = fft_subset_inds[0]

    return [(
        (i - (vec_length * (i // vec_length)) ) # remove channel offset of index
        + vec_start # add the start index of the fft vector subset
        + (i // vec_length) * fft_length) # recover channel offset of original fft vector
        for i in feature_list]

In [25]:
def get_df_from_pb_combos(pb_combos):
    tmp = [[list(sub_ele) for sub_ele in ele] for ele in pb_combos]
    tmp2 = []
    for ele in tmp:
        if len(ele) <= 4:
            _ = [ele.append([None,None]) for i in range(4-len(ele))]
            tmp2.append(ele)
        else:
            [tmp2.append(sub_ele) for sub_ele in ele]
    return pl.DataFrame(tmp2, schema=['PB1', 'PB2', 'PB3', 'PB4'])

In [26]:
def get_training_data_for_LDA(df_fft, pbs, update_rate, label_col='SleepStageBinary'):
   simulated_ffts = (
    df_fft
    .select(
            [
                pl.col('fft_vec').arr.slice(pbs[i][0], (pbs[i][1] - pbs[i][0] + 1)).arr.sum().alias(f'Power_Band{i+1}') 
                for i in range(len(pbs)) if pbs[i][0] is not None
            ] + 
            [pl.col(label_col)]
    )
    .with_row_count()
    .with_columns([
        pl.col('row_nr') // update_rate
    ])
    .groupby(['row_nr'])
    .agg(
        [
            pl.col(f'Power_Band{i+1}').mean() 
            for i in range(len(pbs)) if pbs[i][0] is not None
        ] + 
        [pl.col(label_col).last().alias(label_col)]
      )
   )
   
   X = simulated_ffts.select(pl.col("^Power_Band.*$")).to_numpy()
   y = simulated_ffts.select(pl.col(label_col)).to_numpy().squeeze()
   return X, y
    

In [27]:
# roc_auc_scorer = make_scorer(
#     roc_auc_score, greater_is_better=True, needs_threshold=True
# )

# balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)

def custom_scorer(clf, X, y):
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred, normalize='true')
    acc = accuracy_score(y, y_pred)
    # Leads to different AUC score if simply calling string 'roc_auc' in cross_validate 
    # Maybe because this is because the calling the string 'roc_auc' will use the default scorer,
    #  which only returns the scores on the test sets in the cross-validation?
    auc = roc_auc_score(y, y_pred)
    bal_acc = balanced_accuracy_score(y, y_pred)
    # return {'tn': cm[0, 0], 'fp': cm[0, 1], 'fn': cm[1, 0], 'tp': cm[1, 1],
    #         'accuracy': acc, 'auc': auc, 'balanced_accuracy': bal_acc}
    return {'accuracy': acc, 'auc': auc, 'balanced_accuracy': bal_acc}

In [28]:
def confusion_matrix_scorer(clf, X, y):
    y_pred = clf.predict(X)
    cm = confusion_matrix(y, y_pred, normalize='true')
    return {'tn': cm[0, 0], 'fp': cm[0, 1], 
            'fn': cm[1, 0], 'tp': cm[1, 1]}

In [70]:

def hyperparameter_search_pipeline(device, parameters, sleep_stage_mapping, np_random_seed=0):
    assert len(parameters['TD_Columns']) > 1, 'Must select at least 2 TD channels to run hyperparameter search pipeline.'

    print('Executing Device: ', device)
    np.random.seed(np_random_seed)
    con = duckdb.connect(database='/media/shortterm_ssd/Clay/databases/duckdb/rcs-db.duckdb', read_only=True)
    
    df = get_device_as_pl_df(device, con, lazy=True)
    ALL_TD_COLUMNS = ['TD_BG', 'TD_key2', 'TD_key3']
    # Keep only desired TD columns
    df = df.select(pl.all().exclude( list(set(ALL_TD_COLUMNS) - set(parameters['TD_Columns'])) ) )
    print('Analyzing: ' + ', '.join(df.select(pl.col('^TD.*$')).columns))

    sessions = df.select('SessionIdentity').unique().collect().to_dict(
        as_series=False)['SessionIdentity']
    settings = get_settings_for_pb_calcs(device, con, sessions, 'SessionIdentity')
    gains = get_gains_from_settings_dict(
        settings, sessions, 'SessionIdentity', device, con)
    
    gains_tmp = []
    for channel in parameters['TD_Columns']:
        if channel == 'TD_BG':
            gains_tmp.append(gains[0])
        if channel == 'TD_key2':
            gains_tmp.append(gains[1])
        if channel == 'TD_key3':
            gains_tmp.append(gains[2])
    gains = gains_tmp
    print(gains)

    print(settings)
    print('ASSUMING SUBCORTICAL CHANNEL IS CHANNEL 0')
    fft_length = settings['fft_numBins'][0]

    df_chunked = chunk_df_by_timesegment(df)
    print(df_chunked.columns) # here
    print(f'Simulating FFTs for TD chunks of size {settings["fft_size"][0] - settings["fft_size"][0]%250} samples')
    df_chunked = add_simulated_ffts(df_chunked, settings, gains)
    df_chunked = df_chunked.with_columns(pl.col('SleepStage').map_dict(sleep_stage_mapping).alias('SleepStageBinary'))

    NUM_TD_CHANNELS = len(parameters['TD_Columns'])

    df_chunked = df_chunked.collect()

    df_fft_subset, fft_corrs = get_fft_corrs(df_chunked, parameters['fft_subset_inds'], label_col='SleepStageBinary', td_columns=parameters['TD_Columns'])

    
    fft_subset_length = parameters['fft_subset_inds'][1] - parameters['fft_subset_inds'][0]
    # corr_df = pd.DataFrame({'Hz': 0.48*(np.arange(fft_subset_length) + parameters['fft_subset_inds'][0]), 'BG': fft_corrs[:fft_subset_length], 'key2': fft_corrs[fft_subset_length:fft_subset_length*2], 
    #                         'key3': fft_corrs[fft_subset_length*2:]}).melt(id_vars='Hz', value_vars=['BG', 'key2', 'key3'], var_name='key', value_name='corr')
    # corr_chart = alt.Chart(corr_df).mark_line().encode(x='Hz', y='corr', color='key').interactive()

    first_feature_group, X, y = get_train_data_by_corr_threshold(df_fft_subset, fft_corrs, parameters['corr_threshold'], label_col='SleepStageBinary')
    del(df_fft_subset)
    model = LinearDiscriminantAnalysis(solver='svd')
    print('Running Sequential Feature Selector...')
    sfs = SequentialFeatureSelector(model, n_features_to_select=parameters['num_features_to_select'], direction='forward', n_jobs=6, cv=5, scoring=parameters['sfs_scoring'])
    sfs.fit(X, y)


    second_feature_group = first_feature_group[sfs.get_support(indices=True)]
    second_feature_group_original_inds = np.array(recover_original_feature_inds(second_feature_group, parameters['fft_subset_inds'], fft_length)).squeeze()
    pb_combos, pb_chart = get_PB_combinations(second_feature_group_original_inds, max_clusters=parameters['max_clusters'])
    df_pbs_corrected = get_df_from_pb_combos(pb_combos)
    #df_pbs_corrected = recover_original_feature_inds(df_pb_combos, parameters['fft_subset_inds'], fft_length)

    df_pbs_corrected = df_pbs_corrected.join(pl.DataFrame({'UpdateRate': [2, 5, 10, 15, 30]}), how='cross')

    fft_cols = [col for col in df_chunked.columns if 'fft' in col]
    # df_chunked = df_chunked.select([
    #     pl.col('SleepStage'),
    #     pl.col('SleepStageBinary'),
    #     pl.col('fft_BG').arr.concat([pl.col('fft_key2'), pl.col('fft_key3')]).alias('fft_vec')
    # ])
    df_chunked = df_chunked.select([
        pl.col('SleepStage'),
        pl.col('SleepStageBinary'),
        pl.col(fft_cols[0]).arr.concat(pl.col(fft_cols[1:])).alias('fft_vec')
    ])
    try:
        assert fft_length == df_chunked.select(
                pl.col('fft_vec').arr.lengths()
            ).unique().item() / NUM_TD_CHANNELS
    except AssertionError:
        print('fft_length is not equal to the length of the fft_vec column.')
        print('fft_length: ', fft_length)
        print('length of fft_vec column: ', df_chunked.select(
                pl.col('fft_vec').arr.lengths()
            ).unique().item() / NUM_TD_CHANNELS)


    print('Searching over powerband combos...')
    scores = {'test_accuracy': [], 'test_roc_auc': [], 'test_balanced_accuracy': [], 'test_recall': [], 'test_precision': [], 'test_tnr': []}
    scores_stds = {'test_accuracy': [], 'test_roc_auc': [], 'test_balanced_accuracy': [], 'test_recall': [], 'test_precision': [], 'test_tnr': []}
    for i in range(df_pbs_corrected.height):
        pbs = [value for value in df_pbs_corrected.select(pl.exclude('UpdateRate'))[i].to_dicts()[0].values()]
        X, y = get_training_data_for_LDA(df_chunked, pbs, df_pbs_corrected[i,'UpdateRate'], label_col='SleepStageBinary')

        # NOTE: 'roc_auc' gets label predictions with clf.predict_proba(X)[:, 1], which allows more thresholds to be tested. 
        # clf.predict_proba(X)[:, 1] is the probability of the positive class (1). clf.predict_proba(X)[:, 0] is the probability of the negative class (0)
        score_dict = {'accuracy': 'accuracy', 'roc_auc': 'roc_auc', 'balanced_accuracy': 'balanced_accuracy', 'recall': 'recall', 'precision': 'precision'}

        # cv_results = cross_validate(model, X, y, cv=5,
        #                   scoring=custom_scorer, n_jobs=5)
        cv_results = cross_validate(model, X, y, cv=5,
                        scoring=score_dict, n_jobs=5)
        
        [scores[k].extend([np.mean(v)]) for (k, v) in cv_results.items() if k in scores.keys()]
        [scores_stds[k].extend([np.std(v)]) for (k, v) in cv_results.items() if k in scores_stds.keys()]

        tnr = np.array(cv_results['test_balanced_accuracy']) * 2 - cv_results['test_recall']
        scores['test_tnr'].extend([np.mean(tnr)])
        scores_stds['test_tnr'].extend([np.std(tnr)])
        


    df_hyperparams = pl.concat([df_pbs_corrected, pl.DataFrame(scores).rename(
        {'test_accuracy': 'Acc', 'test_roc_auc': 'AUC', 'test_balanced_accuracy': 'BalAcc', 'test_recall': 'TPR', 'test_precision': 'Precision', 'test_tnr': 'TNR'}),
        pl.DataFrame(scores_stds).rename({'test_accuracy': 'Acc_std', 'test_roc_auc': 'AUC_std', 'test_balanced_accuracy': 'BalAcc_std', 'test_recall': 'TPR_std', 'test_precision': 'precision_std', 'test_tnr': 'TNR_std'})
        ], how='horizontal')

    BASE_PATH =  f'/media/shortterm_ssd/Clay/databases/Sleep_10day_with_autonomic/RCS{device}/'
    df_hyperparams.write_parquet(BASE_PATH + 'hyperparameter_search_results_only_cortical.parquet')
    # corr_chart.save(BASE_PATH + 'sleepstage_corr.png')

    # return (device, df_hyperparams, corr_chart.properties(title=f'{device}'), pb_chart.properties(title=f'{device}'))
    return (device, df_hyperparams, pb_chart.properties(title=f'{device}'))


In [71]:
# The rationale for 24 cpus: Max batch is 4 devices, and each device will use 5 cores for the cross-validation steps. 
# Allocating 1 extra process for cross-validation allows for 6 cores per device.
# According to the documentation, num_cpus provides a 'max number' of cpu cores for that remote task.

# Reduced to 12 because I think I overworked the server and it automatically restarted due to overheating or overload lol
@ray.remote(num_cpus=24) 
def hyperparameter_search_wrapper(devices, params, sleep_stage_mapping):
    for device in devices:
        hyperparameter_search_pipeline(device, params, sleep_stage_mapping)

In [72]:
parameters = {'corr_threshold': 0.05, 'num_features_to_select': 35, 'sfs_scoring': 'roc_auc', 'fft_subset_inds':[2,120], 'max_clusters':8, 'TD_Columns': ['TD_key2', 'TD_key3']}
sleep_stage_mapping = {2: 1, 3: 1, 4: 0, 5: 0, 6: 0}

batches = [['02L', '02R', '03L', '03R', '07L'], ['07R', '09L', '09R', '16L', '16R']]
#batches = [['03L', '07R'], ['16L', '16R']]
full_batch = [hyperparameter_search_wrapper.remote(batch, parameters, sleep_stage_mapping) for batch in batches]
execute = ray.get(full_batch)

# TODO: Write a function that takes in list of indicies and returns powerband edges in Hz

(hyperparameter_search_wrapper pid=1361675) Executing Device:  07R
(hyperparameter_search_wrapper pid=1361643) Executing Device:  02L
(hyperparameter_search_wrapper pid=1361675) Analyzing: TD_key2, TD_key3
(hyperparameter_search_wrapper pid=1361643) Analyzing: TD_key2, TD_key3
(hyperparameter_search_wrapper pid=1361675) WARNING: Session numbers provided have different settings.
(hyperparameter_search_wrapper pid=1361675) Settings dataframe:
(hyperparameter_search_wrapper pid=1361675)    SessionIdentity  samplingRate  ...  Power_Band7_bins  Power_Band8_bins
(hyperparameter_search_wrapper pid=1361675) 0     07R_05-13-22           500  ...       18.07-29.79       40.04-62.99
(hyperparameter_search_wrapper pid=1361675) 1     07R_05-12-22           500  ...       18.07-29.79       40.04-62.99
(hyperparameter_search_wrapper pid=1361675) 2     07R_05-20-22           500  ...       18.07-29.79       40.04-62.99
(hyperparameter_search_wrapper pid=1361675) 3     07R_05-24-22           500  ...  

In [75]:
ray.shutdown()

## End of pipeline

In [20]:
%%time

# NOTE: O7 gets bad results because Powerband values were switched in some sessions... need to exclude those sessions for realistic classification results
update_rates = [1, 5, 10, 15, 30]
devices = ['02L', '02R', '03L', '03R', '07L', '07R', '09L', '09R', '16L', '16R']
#devices = ['02L', '02R']
sleep_stage_mapping = {2: 1, 3: 1, 4: 0, 5: 0, 6: 0}
repeat_cv = False # For just calculating accuracy
np.random.seed(seed=1)

for i, device in enumerate(devices):
   print(device)
   if repeat_cv:
      scores = []
      scores_var = []
   else:
      #cv_results = {'test_tn': [], 'test_fp': [], 'test_fn': [], 'test_tp': []}
      #cv_results = {'test_accuracy': [], 'test_auc': [], 'test_balanced_accuracy': []}
      cv_results = {'test_accuracy': [], 'test_roc_auc': [], 'test_balanced_accuracy': [], 'test_recall': [], 'test_precision': []}
   df = get_device_as_pl_df(device, con)

   for update_rate in update_rates:
      data_for_baseline = (df.filter(pl.col('Power_Band8').is_not_null())
                           .with_row_count().with_columns([
                              pl.col('row_nr') // update_rate
                           ])
                           .groupby(['row_nr']).agg([
                              pl.col('Power_Band5').mean().alias('Power_Band5'),
                              pl.col('Power_Band6').mean().alias('Power_Band6'),
                              pl.col('Power_Band7').mean().alias('Power_Band7'),
                              pl.col('Power_Band8').mean().alias('Power_Band8'),
                              pl.col('SleepStage').last().alias('SleepStage')
                           ])
                           )


      # X = PB features
      X = data_for_baseline.select(pl.col('^Power_.*$')).to_numpy()
      # y = sleep stage labels
      y = data_for_baseline.select(
         pl.col('SleepStage').map_dict(sleep_stage_mapping)
      ).to_numpy().squeeze()
      # solver = {‘svd’, ‘lsqr’, ‘eigen’}
      model = LinearDiscriminantAnalysis(solver='svd')

      if repeat_cv:
         # define model evaluation method
         cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
         # evaluate model
         scores_tmp = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=20)
         scores.append(np.mean(scores_tmp))
         scores_var.append(np.std(scores_tmp))
      else:

         cv_results_tmp = cross_validate(model, X, y, cv=5,
                      scoring=['accuracy', 'roc_auc', 'balanced_accuracy', 'recall', 'precision'])

         [cv_results[k].extend([np.mean(v)]) for (k, v) in cv_results_tmp.items() if k in cv_results.keys()]
         
   if repeat_cv:
      if i == 0:
         df_scores = pd.DataFrame({'update_rate': update_rates, 'score': scores, 'score_std': scores_var, 'device': device})
      else:
         df_tmp = pd.DataFrame({'update_rate': update_rates, 'score': scores, 'score_std': scores_var, 'device': device})
         df_scores = pd.concat([df_scores, df_tmp])
   else:
      if i == 0:
         df_scores = pd.DataFrame({'update_rate': update_rates, **cv_results, 'device': device})
      else:
         df_tmp = pd.DataFrame({'update_rate': update_rates, **cv_results, 'device': device})
         df_scores = pd.concat([df_scores, df_tmp])

print(df_scores.head())


02L
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
02R


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
03L


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
03R


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
07L
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
07R
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
09L
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
09R
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
16L
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
16R
yo
yup
yo
yup
yo
yup
yo
yup
yo
yup
   update_rate  test_accuracy  test_roc_auc  test_balanced_accuracy  \
0            1       0.773409      0.855328                0.751475   
1            5       0.839304      0.910213                0.831211   
2           10       0.859093      0.928028                0.853971   
3           15       0.866331      0.935618                0.862471   
4           30       0.877015      0.943806                0.873753   

   test_recall  test_precision device  
0     0.892940        0.757882    02L  
1     0.883538        0.845184    02L  
2     0.887112        0.871150    02L  
3     0.887338        0.882018    02L  
4     0.894749        0.892622    02L  
CPU times: user 14min 20s, sys: 23min 50s, total: 38min 10s
Wall

In [13]:
def custom_score_dict(clf, X, y):
    y_pred = clf.predict(X)
    acc = accuracy_score(y, y_pred)
    auc = roc_auc_score(y, clf.predict_proba(X)[:, 1])
    bal_acc = balanced_accuracy_score(y, y_pred)
    return {'accuracy': acc, 'roc_auc': auc, 'balanced_accuracy': bal_acc}

X = np.random.rand(1000, 2)
y = np.random.randint(0, 2, 1000)
model = LinearDiscriminantAnalysis(solver='svd')
cv_results = cross_validate(model, X, y, cv=5,
                      scoring=custom_score_dict)
print('custom')
print(cv_results)

cv_results_comparison = cross_validate(model, X, y, cv=5,
            scoring=['accuracy', 'roc_auc', 'balanced_accuracy'])
print('list')
print(cv_results_comparison)

custom
{'fit_time': array([0.00213289, 0.00131989, 0.00129437, 0.0012908 , 0.00128198]), 'score_time': array([0.00290585, 0.00214028, 0.00209594, 0.00210071, 0.00208092]), 'test_accuracy': array([0.52 , 0.52 , 0.52 , 0.5  , 0.515]), 'test_roc_auc': array([0.49659455, 0.52483974, 0.4931891 , 0.48003203, 0.48343509]), 'test_balanced_accuracy': array([0.5       , 0.5       , 0.5       , 0.48813933, 0.50120108])}
list
{'fit_time': array([0.00129676, 0.00128746, 0.0012908 , 0.00127912, 0.00127935]), 'score_time': array([0.00229549, 0.00209332, 0.00206327, 0.00211072, 0.00208974]), 'test_accuracy': array([0.52 , 0.52 , 0.52 , 0.5  , 0.515]), 'test_roc_auc': array([0.49659455, 0.52483974, 0.4931891 , 0.48003203, 0.48343509]), 'test_balanced_accuracy': array([0.5       , 0.5       , 0.5       , 0.48813933, 0.50120108])}


In [25]:

px.line(df_scores, x='update_rate', y='score', error_y='score_std', color='device', labels={'x': 'Update rate', 'y': 'Accuracy'})


FigureWidgetResampler({
    'data': [{'error_y': {'array': array([0.00166065, 0.00384584, 0.00513859, 0.006003…

In [19]:
px.line(df_scores, x='update_rate', y='test_tp', color='device', labels={'x': 'Update rate', 'y': 'True Positive Rate'})


FigureWidgetResampler({
    'data': [{'hovertemplate': 'device=02L<br>update_rate=%{x}<br>test_tp=%{y}<extra><…

In [20]:
px.line(df_scores, x='update_rate', y='test_tn', color='device', labels={'x': 'Update rate', 'y': 'True Negative Rate'})

FigureWidgetResampler({
    'data': [{'hovertemplate': 'device=02L<br>update_rate=%{x}<br>test_tn=%{y}<extra><…

In [23]:
df_scores[df_scores['update_rate'] == 30]

,update_rate,test_accuracy,test_roc_auc,test_balanced_accuracy,test_recall,test_precision,device
4,30,0.877015,0.943806,0.873753,0.894749,0.892622,02L
4,30,0.869398,0.935008,0.868966,0.871146,0.907149,02R
4,30,0.866339,0.935992,0.867729,0.847119,0.896761,03L
4,30,0.854151,0.934989,0.856023,0.794945,0.910582,03R
4,30,0.668392,0.760244,0.620820,0.901478,0.658004,07L
4,30,0.703113,0.728040,0.578734,0.980863,0.693077,07R
4,30,0.854953,0.915836,0.851244,0.871200,0.882966,09L
4,30,0.796285,0.844178,0.788826,0.867500,0.783636,09R
4,30,0.789860,0.838137,0.785051,0.815782,0.819893,16L
4,30,0.833746,0.869149,0.820183,0.900363,0.829904,16R


In [80]:
# TODO: Implement a funciton that visualizes the LDA boundary line. Check sleep class directory

## Parallelization Sandbox

In [29]:
def rcssim_wrapper_parallelized(i, key):
    fft, _ = rcssim_wrapper(td_np[i,key], td_np[i,0], sim_settings, gains[key-1])
    return fft

In [ ]:
# Does not speed things up, actually slows it down. Most processes are idle.
fft_arr_BG_tmp = Parallel(n_jobs=20)(delayed(rcssim_wrapper_parallelized)(i, 1) for i in range(td_np.shape[0]))

In [ ]:
from pathos.multiprocessing import ProcessingPool as Pool
pool = Pool(processes=20)
con = None # Not sure why, but pool.map tries to pickle the duckdb connection object, which fails
fft_arr_BG_tmp = pool.map(rcssim_wrapper_parallelized, [(i, 1) for i in range(td_np.shape[0])])

In [27]:
from pathos.multiprocessing import ProcessingPool as Pool
from math import sqrt
pool = Pool(processes=20)
results = pool.map(sqrt, range(1000000))

In [25]:
for i in range(1000000):
    sqrt(i)